In [14]:
import gradio as gr
import clickhouse_connect
from embedding import E5LargeEmbeddingFunction
from similarity import bm25_ensemble_with_crossenc_answer, prep_query, bm25_fewshot_with_cross_encoder
from pprint import pprint
import requests
import json
import pandas as pd
import gc


client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')
emb_func = E5LargeEmbeddingFunction()

def message_llm(system_prompt, user_input, temperature=0.6):
    '''Function to message (test) our LLM'''

    try:
        url = "https://live-relaxed-oryx.ngrok-free.app/v1/chat/completions"

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
          # "max_tokens": -1,
          # "stream": False
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")
        
rag_prompt = """
### CONTEXT ###
1) ИСТОЧНИК 1: 
- {type_of_source1}
- {link_to_source1}
- {batch_1}

2) ИСТОЧНИК 2: 
- {type_of_source2}
- {link_to_source2}
- {batch_2}

3) ИСТОЧНИК 3:
- {type_of_source3}
- {link_to_source3}
- {batch_3}

### INSTRUCTION ###
На основании приведенных данных категории «ТЕКСТ» дай исчерпывающий ответ на
приведенный ниже вопрос:
- {QUESTION}
"""

system_prompt = """
### ROLE ###
Ты - дружелюбный ассистент Центрального банка Российской Федерации. 
Исчерпывающе отвечай клиентам на возникшие вопросы. Если из контекста 
невозможно дать ответ на вопрос, пиши «Я не могу ответить на этот вопрос»."""

def ask_chatbot(question):
    gc.collect()
    query = prep_query(question)
    
    result = bm25_ensemble_with_crossenc_answer(query=query, client=client, emb_func=emb_func, bm25_n_results=15, cr_enc_n_results=3, limit_knn=100, knn_docs_window=1)

    type_of_source1, type_of_source2, type_of_source3  = '', '', ''
    link_to_source1, link_to_source2, link_to_source3 = '', '', ''
    batch_1, batch_2, batch_3 = '', '', ''


    for idx, batch in enumerate(start=1, iterable=result):
        if idx==1:
            if batch[5]:
                type_of_source1 = 'ТАБЛИЦА'
            else:
                type_of_source1 = 'ТЕКСТ'
            link_to_source1 = batch[2]
            batch_1 = batch[3]

        if idx==2:
            if batch[5]:
                type_of_source2 = 'ТАБЛИЦА'
            else:
                type_of_source2 = 'ТЕКСТ'
            link_to_source2 = batch[2]
            batch_2 = batch[3]

        if idx==3:
            if batch[5]:
                type_of_source3 = 'ТАБЛИЦА'
            else:
                type_of_source3 = 'ТЕКСТ'
            link_to_source3 = batch[2]
            batch_3 = batch[3]


    formatted_rag_prompt = rag_prompt.format(
        type_of_source1=type_of_source1,
        link_to_source1=link_to_source1,
        batch_1=batch_1,
        type_of_source2=type_of_source2,
        link_to_source2=link_to_source2,
        batch_2=batch_2,
        type_of_source3=type_of_source3,
        link_to_source3=link_to_source3,
        batch_3=batch_3,
        QUESTION=question
    )

    answer = message_llm(system_prompt, formatted_rag_prompt)
    
    return f"{answer}\nИсточники: \n{link_to_source1}, \n{link_to_source2}, \n{link_to_source3}"

iface = gr.Interface(
    fn=ask_chatbot,
    inputs="text",
    outputs="text",
    title="Чат с виртуальным помощником - SCOOF DEVELOPERS",
    description="Введите ваше сообщение и получите ответ от модели:"
)

# Запуск интерфейса
iface.launch()


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
